In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from time import sleep
import json
import os
import subprocess
import wget

my_options = webdriver.ChromeOptions()
# my_options.add_argument("--headless")
my_options.add_argument("--start-maximized")
my_options.add_argument("--incognito")
my_options.add_argument("--disable-popup-blocking")
my_options.add_argument("--disable-notifications")
my_options.add_argument("--lang=zh-TW")
driver = webdriver.Chrome(
    options = my_options
)

folderPath = 'youtube'
if not os.path.exists(folderPath):
    os.mkdir(folderPath)

listData = []

In [2]:
def visit():
    driver.get('https://www.youtube.com/')

def search():
    txtInput = driver.find_element(
        By.CSS_SELECTOR, "input#search"
    )
    txtInput.send_keys('烏薩奇')
    
    sleep(1)

    txtInput.submit()

    sleep(1)

def filterFunc():
    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located(
                (By.CSS_SELECTOR,'ytd-button-renderer.style-scope.ytd-search-header-renderer')
            )
        )

        driver.find_element(
            By.CSS_SELECTOR,'ytd-button-renderer.style-scope.ytd-search-header-renderer'
        ).click()

        sleep(2)

        driver.find_elements(
            By.CSS_SELECTOR,'div#options a#endpoint.yt-simple-endpoint.style-scope.ytd-search-filter-renderer'
        )[9].click()

        sleep(2)

    except TimeoutException:
        print('等待逾時')

def scroll():
    innerHeight = 0
    offset = 0
    count = 0
    limit = 3

    while count < limit:
        offset = driver.execute_script(
            'return document.documentElement.scrollHeight;'
        )

        driver.execute_script(f'''
            window.scrollTo({{
                top: {offset},
                behavior: 'smooth'
            }})
        ''')

        break 
        # 只捲動一次
        # sleep(3)

        # innerHeight = driver.execute_script(
        #     'return document.documentElement.scrollHeight;'
        # )

        # if offset == innerHeight:
        #     count += 1

def parse():
    global listData
    listData.clear()

    elements = driver.find_elements(
        By.CSS_SELECTOR, 
        'ytd-video-renderer.style-scope.ytd-item-section-renderer'
    )

    for elm in elements:
        print("=" * 30)

        img = elm.find_element(By.CSS_SELECTOR, "a#thumbnail img")
        imgSrc = img.get_attribute('src')
        print(imgSrc)

        a = elm.find_element(By.CSS_SELECTOR, "a#video-title")
        aTitle = a.get_attribute('innerText')
        print(aTitle)

        aLink = a.get_attribute('href')
        print(aLink)

        strDelimiter = ''
        if 'shorts' in aLink:
            strDelimiter = '/shorts/'
        else:
            strDelimiter = 'v='
        youtube_id = aLink.split(strDelimiter)[1]
        youtube_id = youtube_id.split('&pp')[0]
        print(youtube_id)

        listData.append({
            "id": youtube_id,
            "title": aTitle,
            "link": aLink,
            "img": imgSrc
        })

def saveJson():
    with open(f'{folderPath}/youtube.json', 'w', encoding='utf-8') as file:
        file.write(
            json.dumps(
                listData,
                ensure_ascii=False,
                indent=4
            )
        )

def close():
    driver.quit()

In [3]:
visit()
search()
filterFunc()
scroll()
parse()
saveJson()

https://i.ytimg.com/vi/BLeZ9r0rJIQ/hqdefault.jpg?sqp=-oaymwEnCOADEI4CSFryq4qpAxkIARUAAIhCGAHYAQHiAQoIGBACGAY4AUAB&rs=AOn4CLBzZGgwoefGVCwF9KETg1PPEF-62g
Chiikawa Usagi 吉伊卡哇 烏薩奇 蛤 合輯 11種
https://www.youtube.com/watch?v=BLeZ9r0rJIQ&pp=ygUJ54OP6Jap5aWH
BLeZ9r0rJIQ
https://i.ytimg.com/vi/ivIl7PD0Wvo/hq720.jpg?sqp=-oaymwFBCNAFEJQDSFryq4qpAzMIARUAAIhCGAHYAQHiAQoIGBACGAY4AUAB8AEB-AH-CYAC0AWKAgwIABABGGUgZShlMA8=&rs=AOn4CLAjjTcRWVOeUoOtxk_Ytmv9Mogg5w
烏薩奇颱風
https://www.youtube.com/watch?v=ivIl7PD0Wvo&pp=ygUJ54OP6Jap5aWH
ivIl7PD0Wvo
https://i.ytimg.com/vi/Kp9YVbAS4g4/hq720.jpg?sqp=-oaymwFBCNAFEJQDSFryq4qpAzMIARUAAIhCGAHYAQHiAQoIGBACGAY4AUAB8AEB-AH4CYAC0AWKAgwIABABGFMgYChlMA8=&rs=AOn4CLAvHIzv0AjiDVxjnpYry3SYNf82Mw
Usagi Singing Collection 『うさぎ』歌う “ 乌萨奇唱歌合集”(中日双语字幕）
https://www.youtube.com/watch?v=Kp9YVbAS4g4&pp=ygUJ54OP6Jap5aWH
Kp9YVbAS4g4
https://i.ytimg.com/vi/Y4b-AWTOjmQ/hq720.jpg?sqp=-oaymwFBCNAFEJQDSFryq4qpAzMIARUAAIhCGAHYAQHiAQoIGBACGAY4AUAB8AEB-AH-CYAC0AWKAgwIABABGGUgTChQMA8=&rs=AOn4CLD7MSPg

In [4]:
close()

In [5]:
def download():
    if not os.path.exists('./yt-dlp.exe'):
        print('[下載 yt-dlp]')
        wget.download('https://github.com/yt-dlp/yt-dlp/releases/latest/download/yt-dlp.exe', './yt-dlp.exe')

    with open(f'{folderPath}/youtube.json', 'r', encoding='utf-8') as file:
        strJson = file.read()
        listResult = json.loads(strJson)

    for index, obj in enumerate(listResult):
        if index == 3:
            break

        print(f'正在下載:{obj["link"]}')
        cmd = [
            './yt-dlp.exe',
            '-i',
            obj['link'],
            '-f', 'b[ext=mp4]',
            '-o', f'{folderPath}/%(id)s.%(ext)s'
        ]
        obj_sp = subprocess.run(cmd)

        if obj_sp.returncode == 0:
            print('[下載成功]')
        else:
            print('[下載失敗]')


download()

正在下載:https://www.youtube.com/watch?v=BLeZ9r0rJIQ&pp=ygUJ54OP6Jap5aWH
[下載成功]
正在下載:https://www.youtube.com/watch?v=ivIl7PD0Wvo&pp=ygUJ54OP6Jap5aWH
[下載成功]
正在下載:https://www.youtube.com/watch?v=Kp9YVbAS4g4&pp=ygUJ54OP6Jap5aWH
[下載成功]
